In [ ]:
#채팅서버
import socket, threading
members=[]

def sendMsgAll(msg):#인자로 받은 메시지를 채팅방에 있는 모든 사람에 전송
    print('sendMsgAll:', msg)
    for i in members:
        i.sendall(msg.encode(encoding='utf-8'))
            
#쓰레드 작업. 클라이언트 하나당 하나 생성됨. 소켓으로 연결된 클라이언트가 전송한 메시지를 받아서 전체에 전달
def f1(p, soc):
    id = 'member'+str(p)
    msg = id+'님이 입장하셨습니다'
    sendMsgAll(msg)
    
    while True:
        msg = soc.recv(100)
        msg = msg.decode()
        if msg=='/end':
            msg = id+'님이 퇴장하셨습니다'
            soc.sendall('/end'.encode(encoding='utf-8'))
            members.remove(soc)#채팅방에서 제거
            sendMsgAll(msg)
            break
        msg = id+': '+msg
        sendMsgAll(msg)
    
def main():
    host = '172.30.1.17' #서버 주소
    port = 3234             #서버 포트번호

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind((host, port))
    server_socket.listen()
    print('chatting server start')
    
    while True:
        client_soc, addr = server_socket.accept()#클라이언트 기다림
        members.append(client_soc)
        print('connected client addr:', addr)
        #쓰레드 생성
        th = threading.Thread(target=f1, args=(addr[1], client_soc))
        th.start()
    
    print('서버 종료')
    server_socket.close()
    
main()

chatting server start
connected client addr: ('172.30.1.17', 3452)
sendMsgAll: member3452님이 입장하셨습니다
sendMsgAll: member3452: sdflkj
sendMsgAll: member3452: dlk
sendMsgAll: member3452: wo
sendMsgAll: member3452: 뭔가 이상항
sendMsgAll: member3452: 이게 왜 스레딩이 이렇냐
sendMsgAll: member3452님이 퇴장하셨습니다


#### 채팅 클라이언트
- 스레드로 작업을 분리
- th1: 서버가 전송한 메시지 받아서 출력
- th2: 메시지를 키보드 입력받아서 서버로 전송.

In [ ]:
import socket, threading

soc = None

def sendMsg():
    while True:
        msg = input('msg:')
        soc.sendall(msg.encode(encoding='utf-8'))
        if msg == '/end':
            break


def recvMsg():
    while True:
        data = soc.recv(100)
        msg = data.decode()  # 읽은 데이터 디코딩
        print(msg)
        if msg == '/end':
            break
    soc.close()


def main():
    global soc
    server_ip = '172.30.1.17'
    server_port = 3234

    soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    soc.connect((server_ip, server_port))

    th1 = threading.Thread(target=sendMsg)
    th1.start()

    th2 = threading.Thread(target=recvMsg)
    th2.start()


main()